# Physics example 1: classification of SUSY signal over SM background events

In the first example we look at building a multi-layered dense neural network with tensorflow to classify SUSY signal events over background events.

Further down, in 'Physics example 2', we look at the example of a simple regression test. 

In [ ]:
#import all the required libraries here
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import model_selection,preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

### First load the dataset and define the variables to train over

We will use the popular python analysis library pandas to load and manipulate the data within a dataframe.

The dataset we will use has 50K events, 25K of each signal and background. This is indicated by the column 'signal' (0 for background, 1 for signal). Several high and low level physics variables have been precalculated and stored, we will use a typical subset used in SUSY analyses as features for the network to train on.

The background consists of ttbar events and the signal consists of the T2tt simplified SUSY model, where stops are pair produced and decay to a weakly interacting LSP and a top quark. For this particular point the mass of the stop is 900 GeV and the mass of the LSP is 100 GeV

In [ ]:
dfFull = pd.read_pickle('data/stop50K.pkl')

#Pick a subset of features from the dataframe
subset=['signal', #1 for signal and 0 for background
        'HT','MET', #energy sums
        'MT','MT2W', #topological variables
        'n_jet','n_bjet', #jet and b-tag multiplicities
        'sel_lep_pt0','sel_lep_eta0','sel_lep_phi0', #lepton 4-vector
        'selJet_phi0','selJet_pt0','selJet_eta0','selJet_m0',# lead jet 4-vector
        'selJet_phi1','selJet_pt1','selJet_eta1','selJet_m1',# second jet 4-vector
        'selJet_phi2','selJet_pt2','selJet_eta2','selJet_m2']# third jet 4-vector

df = dfFull[subset]

print 'The df has',(df.signal==1).sum(),'signal events and',(df.signal==0).sum(),'background events'
print '\nSummary statistics:'
df.describe()

### Preparing the data for a neural network training: splitting and standardising

To prepare the data for machine learning with a neural network we need to split into a testing and training set, to avoid bias when evaluating the performance. We additionally adopt the naming convention of 'y' for target variables and 'X' for input variables.

It is also sensible to 'standardise' the input variables for a neural network by scaling them to have a standard deviation between -1 and 1 and a mean of 0. This means all the features have the same effective weight when training the network. 

For these task we use functions from the useful python ML library 'scikit-learn'.

In [ ]:
X=df.drop('signal',axis=1)
y=df['signal']#.values.reshape(len(df),1)

#Split with 30% of data kept aside for testing
X_train,X_test,y_train,y_test = model_selection.train_test_split(X, y, test_size=0.3, 
                                                                 random_state=42)

#Now do the standardisation on the train set
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train),columns=X_train.columns,index=X_train.index)

#Additionally apply the scaling it to the test set
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns,index=X_test.index)

#Now check it worked
print 'Standardised statistics:'
X_train.describe()

### Building the neural network model

Now we have a prepared dataset ready for our neural network we need to build the model with tensorflow. Working from the example from 'gaussian.ipynb', build a 2 hidden layer network with 50 nodes on each layer. Therefore requiring 3 sets of weight and biases. We will choose the activation function of the internal nodes to be a RELU.

The input size will be the number of features available in X and the output will be one node with a sigmoid function to choose between 1 or 0 (signal or background).

In [ ]:
def model(x,input_size,layer_size=50,sigmoid_out=True,scope_name='model'):
    #This ensuring the variables are defined within the scope of the model
    
    ###### YOUR CODE HERE ######

    return logits, tf.sigmoid(logits)
    

#Define the placeholder for the input

###### YOUR CODE HERE ######
# x = tf.placeholder(...)
# logits,f = model(...)

### Add the training operations

As in the gaussian example we want to define the outputs to train to and use the binary cross entropy loss function.

In [ ]:
###### YOUR CODE HERE ######

### Run the training

Compile the model into a session and run the training as it has been setup, as in the gaussian example.

However, in this case we are going to train over mini-batches of 1024 events each step.

In [ ]:
#Set up and initialise the session
# sess = ....
###### YOUR CODE HERE ######

nEpochs=10
batch_size=1024

#Loop over epochs but this 
# I'll leave the code in place for you
# Note that we do an inner loop on each epoch selecting a batch of events
# We also have to make sure to format the pandas input properly with np.vstack
for i in range(nEpochs):
    loss_batch=[]
    for start,end in zip(range(0,len(X_train),batch_size),
                         range(batch_size,len(X_train)+1,batch_size)):
        loss_,_ = sess.run([loss,minimize_loss],
                          #feed_dict={x: X_train[start:end], labels: y_train[start:end]})
                           feed_dict={x: np.vstack(X_train.as_matrix())[start:end], labels: np.vstack(y_train)[start:end]})
        loss_batch.append(loss_)
        #print loss_
    #print loss_batch
    loss_ = np.mean(loss_batch)
    loss_train.append(loss_)
    print 'Epoch',i,';',
    print 'Train loss',loss_,';',
    loss_ = sess.run(loss, feed_dict={x: np.vstack(X_test.as_matrix()), labels: np.vstack(y_test)})
    loss_test.append(loss_)
    print 'Test loss',loss_

In [ ]:
plt.plot(range(1, len(loss_train)+1), loss_train, lw=3, label="Training loss")
plt.plot(range(1, len(loss_train)+1), loss_test, lw=3, label="Testing loss")
plt.legend()
plt.xlabel("Gradient step"), plt.ylabel("Cross-entropy loss");

### Now check the performance

Check for over training by plotting the decision functions for test and train.

Check classification performance by plotting a ROC curve.

In [ ]:
pred_train_sig = sess.run(f, feed_dict={x: X_train[y_train==1]})
pred_train_bkgd = sess.run(f, feed_dict={x: X_train[y_train==0]})
pred_test_sig = sess.run(f, feed_dict={x: X_test[y_test==1]})
pred_test_bkgd = sess.run(f, feed_dict={x: X_test[y_test==0]})


plt.hist(pred_train_sig,bins=50,range=(0.,1.),label='Train prediction sig',density=True,histtype='step')
plt.hist(pred_train_bkgd,bins=50,range=(0.,1.),label='Train prediction bkgd',density=True,histtype='step')
plt.hist(pred_test_sig,bins=50,range=(0.,1.),label='Train prediction sig',density=True,histtype='step',linestyle='dashed')
plt.hist(pred_test_bkgd,bins=50,range=(0.,1.),label='Train prediction bkgd',density=True,histtype='step',linestyle='dashed')


#plt.hist(pred_test,bins=50,range=(0,1),label='Test prediction',density=True,histtype='step')
plt.legend(loc='upper left')
plt.show()

If everything looks good, move on to the ROC curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, sess.run(f, feed_dict={x:X_test}))
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, lw=1, label='(area = %0.2f)'%(roc_auc))
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.grid()   

# Physics example 2: regression to calculate MT

Further down, in 'Physics example 2', we look at the example of a simple regression test. 

Try to teach the network to learn M_T, the transverse mass of the lepton and MET. We will just do this with the signal dataset.

Again start with defining a subset of required variables and splitting.

In [ ]:
subset=[
        'MT',
        'MET','METPhi',
        'sel_lep_pt0','sel_lep_eta0','sel_lep_phi0' #lepton 4-vector
        ]

dfR = dfFull[(dfFull.signal==1)][subset]

XR=dfR.drop('MT',axis=1)
yR=dfR['MT']#.values.reshape(len(df),1)

#Split with 30% of data kept aside for testing
XR_train,XR_test,yR_train,yR_test = model_selection.train_test_split(XR, yR, test_size=0.3, 
                                                                 random_state=42)

### Building the neural network model

Build a two layer model as above, but this time the output layer should remain linear instead of sigmoid and we reduce the number of nodes per layer. You can modify the 'model' function or rewrite the function.

In [ ]:
###### YOUR CODE HERE ######
# xR = ...
# logitsR,fR = model(....)

### Define and run the training

This time we use mean squared error as a loss function

In [ ]:
###### YOUR CODE HERE ######
#labelsR = tf.placeholder(...)

lossR= tf.reduce_mean(tf.losses.mean_squared_error(labels=labelsR,predictions=fR))
#minimize_lossR = ...

In [ ]:
###### YOUR CODE HERE ######
# Define and initialise sessio

# Run, looping over epochs and batch as in the 6th cell above

In [ ]:
plt.plot(range(1, len(lossR_train)+1), lossR_train, lw=3, label="Training loss")
plt.plot(range(1, len(lossR_train)+1), lossR_test, lw=3, label="Testing loss")
plt.legend()
plt.xlabel("Gradient step"), plt.ylabel("Mean squared error loss");

### Now make some output

This time just plot the predicted vs the true M_T for the training set

In [ ]:
predR_test = sessR.run(fR, feed_dict={xR: XR_test})

plt.hist(predR_test,bins=100,range=(0,1000),label='Prediction',alpha=0.7)
plt.hist(yR_test,bins=100,range=(0,1000),label='Truth',alpha=0.7)
plt.legend()
plt.show()

How does it look? Can you tweak the model to improve performance?